In [568]:
import pygame
import sys
import random

In [569]:
starting_board = [
    ["r", "n", "b", "q", "k", "b", "n", "r"],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    [" ", " ", " ", " ", " ", " ", " ", " "],
    ["BP", "BP", "BP", "BP", "BP", "BP", "BP", "BP"],
    ["BR", "BN", "BB", "BQ", "BK", "BB", "BN", "BR"]
]

board = [row[:] for row in starting_board]

WIDTH, HEIGHT = 512, 512
DIMENSION = 8
SQUARE_SIZE = HEIGHT // DIMENSION


LIGHT_SQUARE = (240, 217, 181)
DARK_SQUARE = (181, 136, 99)

def draw_board(screen):
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            color = LIGHT_SQUARE if (row + col) % 2 == 0 else DARK_SQUARE
            pygame.draw.rect(screen, color, pygame.Rect(col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

In [570]:
def place_piece(screen, piece_image_file, row, col):
    img = pygame.image.load(piece_image_file)
    img = pygame.transform.scale(img, (SQUARE_SIZE, SQUARE_SIZE))
    screen.blit(img, pygame.Rect(col * SQUARE_SIZE, (7-row) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

In [571]:
def is_invalid_move(board, start_row, start_col, end_row, end_col, current_player):
    if start_row == end_row and start_col == end_col:
        print("Başlangıç ve bitiş pozisyonları aynı, geçersiz hamle.")
        return True

    if board[end_row][end_col] != ' ' and board[end_row][end_col].islower() == current_player.islower():
        print("Hedef karede aynı renkten bir taş var, geçersiz hamle.")
        return True
    if board[start_row][start_col].lower() != current_player:
        print("Başlangıç karesinde kendi renginizden bir taş yok, geçersiz hamle.")
        return True

    return False

--------------------------------------------------------------------------------------------------------Pawn's Movenement Rules-------------------------------------------------------------

In [572]:
def promote_pawn(board, row, col):
    player = board[row][col].lower()
    if (player == 'w' and row == 0) or (player == 'b' and row == 7):
        print("Piyon tahtanın sonuna ulaştı! Lütfen bir taş seçin: (v, a, k, f)")
        while True:
            piece_type = input().lower()
            if piece_type in ['v', 'a', 'k', 'f']:
                if player == 'w':
                    piece_type = piece_type.upper()
                board[row][col] = piece_type
                break
            else:
                print("Geçersiz taş! Lütfen v, a, k veya f girin.")

In [573]:
def is_square_attacked(board, row, col, player):
    for i in range(8):
        if board[row][i] != ' ' and board[row][i].lower() != player:
            pygame.display.flip()
            return True
        if board[i][col] != ' ' and board[i][col].lower() != player:
            pygame.display.flip()
            return True

    for i in range(1, 8):
        if row - i >= 0 and col + i < 8:
            if board[row - i][col + i] != ' ' and board[row - i][col + i].lower() != player:
                pygame.display.flip()
                return True
        if row - i >= 0 and col - i >= 0:
            if board[row - i][col - i] != ' ' and board[row - i][col - i].lower() != player:
                pygame.display.flip()
                return True
        if row + i < 8 and col + i < 8:
            if board[row + i][col + i] != ' ' and board[row + i][col + i].lower() != player:
                pygame.display.flip()
                return True
        if row + i < 8 and col - i >= 0:
            if board[row + i][col - i] != ' ' and board[row + i][col - i].lower() != player:
                pygame.display.flip()
                return True

    knight_moves = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (-1, 2), (1, -2), (-1, -2)]
    for dr, dc in knight_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'n' and board[row + dr][col + dc].islower() != player.islower():
                pygame.display.flip()
                return True

    rook_directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for dr, dc in rook_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'r' and board[r][c].islower() != player.islower():
                    pygame.display.flip()
                    return True
                else:
                    break

    bishop_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    for dr, dc in bishop_directions:
        r, c = row, col
        while 0 <= r + dr < 8 and 0 <= c + dc < 8:
            r += dr
            c += dc
            if board[r][c] != ' ':
                if board[r][c].lower() == 'b' and board[r][c].islower() != player.islower():
                    return True
                else:
                    break
                
    king_moves = [(1, 1), (1, 0), (1, -1), (0, 1), (0, -1), (-1, 1), (-1, 0), (-1, -1)]
    for dr, dc in king_moves:
        if 0 <= row + dr < 8 and 0 <= col + dc < 8:
            if board[row + dr][col + dc].lower() == 'k' and board[row + dr][col + dc].islower() != player.islower():
                return True

    return False


def is_valid_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible):
    piece = board[start_row][start_col]
    target = board[end_row][end_col]

    
    if (player == 'w' and not piece.startswith('B')) or (player == 'b' and piece.startswith('B')):
        return False
    
    if target != ' ':
        if player == 'w' and target.isupper():
            return False
        if player == 'b' and target.islower():
            return False    
    piece = piece[1:] if piece.startswith('B') else piece
    
    if piece.lower() == 'p':
        return is_valid_pawn_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible)
    elif piece.lower() == 'r':
        return is_valid_rook_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'n':
        return is_valid_knight_move(start_row, start_col, end_row, end_col)
    elif piece.lower() == 'b':
        return is_valid_bishop_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'q':
        return is_valid_queen_move(board, start_row, start_col, end_row, end_col)
    elif piece.lower() == 'k':
        return is_valid_king_move(start_row, start_col, end_row, end_col) or is_valid_castling(board, start_row, start_col, end_row, end_col, player)
    
    return False

def is_valid_pawn_move(board, start_row, start_col, end_row, end_col, player, en_passant_possible):
    direction = -1 if player == 'w' else 1
    
    if start_col == end_col and end_row == start_row + direction and board[end_row][end_col] == ' ':
        return True
    
    if start_col == end_col and end_row == start_row + 2 * direction and board[end_row][end_col] == ' ' and board[start_row + direction][start_col] == ' ':
        return (player == 'w' and start_row == 6) or (player == 'b' and start_row == 1)
    
    if abs(start_col - end_col) == 1 and end_row == start_row + direction:
        if player == 'w' and not board[end_row][end_col].startswith('B') and board[end_row][end_col] != ' ':
            return True
        if player == 'b' and board[end_row][end_col].startswith('B'):
            return True
    
    if en_passant_possible and abs(start_col - end_col) == 1 and end_row == start_row + direction:
        if player == 'w' and start_row == 3 and board[start_row][end_col] == 'p':
            return True
        if player == 'b' and start_row == 4 and board[start_row][end_col] == 'P':
            return True
    
    return False

def is_valid_rook_move(board, start_row, start_col, end_row, end_col):
    if start_row != end_row and start_col != end_col:
        return False
    
    row_direction = 0 if start_row == end_row else (1 if end_row > start_row else -1)
    col_direction = 0 if start_col == end_col else (1 if end_col > start_col else -1)
    
    current_row, current_col = start_row + row_direction, start_col + col_direction
    while (current_row, current_col) != (end_row, end_col):
        if board[current_row][current_col] != ' ':
            return False
        current_row += row_direction
        current_col += col_direction
    
    return True

def is_valid_knight_move(start_row, start_col, end_row, end_col):
    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)
    return (row_diff == 2 and col_diff == 1) or (row_diff == 1 and col_diff == 2)

def is_valid_bishop_move(board, start_row, start_col, end_row, end_col):
    if abs(end_row - start_row) != abs(end_col - start_col):
        return False
    
    row_direction = 1 if end_row > start_row else -1
    col_direction = 1 if end_col > start_col else -1
    
    current_row, current_col = start_row + row_direction, start_col + col_direction
    while (current_row, current_col) != (end_row, end_col):
        if board[current_row][current_col] != ' ':
            return False
        current_row += row_direction
        current_col += col_direction
    
    return True

def is_valid_queen_move(board, start_row, start_col, end_row, end_col):
    return is_valid_rook_move(board, start_row, start_col, end_row, end_col) or is_valid_bishop_move(board, start_row, start_col, end_row, end_col)

def is_valid_king_move(start_row, start_col, end_row, end_col):
    row_diff = abs(end_row - start_row)
    col_diff = abs(end_col - start_col)
    return row_diff <= 1 and col_diff <= 1

def is_valid_castling(board, start_row, start_col, end_row, end_col, player):
    if board[start_row][start_col].lower() != 'k':
        return False
    
    if end_row != start_row:
        return False
    
    if abs(end_col - start_col) != 2:
        return False
    
    if player == 'w':
        row = 7
    else:
        row = 0
    
    if end_col == 6:  # Kingside castling
        if board[row][5] != ' ' or board[row][6] != ' ':
            return False
        if is_square_attacked(board, row, 4, player) or is_square_attacked(board, row, 5, player):
            return False
    elif end_col == 2:  # Queenside castling
        if board[row][1] != ' ' or board[row][2] != ' ' or board[row][3] != ' ':
            return False
        if is_square_attacked(board, row, 2, player) or is_square_attacked(board, row, 3, player):
            return False
    else:
        return False
    
    return True

In [574]:
def make_move(board, screen, start_row, start_col, end_row, end_col):
    piece = board[start_row][start_col]
    board[end_row][end_col] = piece
    board[start_row][start_col] = ' '

    if piece.lower() == 'k' and abs(end_col - start_col) == 2:
        if end_col > start_col:
            rook = board[start_row][7]
            board[start_row][7] = ' '
            board[start_row][5] = rook
        else:
            rook = board[start_row][0]
            board[start_row][0] = ' '
            board[start_row][3] = rook

    draw_board(screen)
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            piece = board[row][col]
            if piece != ' ':
                piece_image_file = piece.upper() + ".png"
                place_piece(screen, piece_image_file, row, col)
    pygame.display.flip()
    
    if is_valid_castling(board, start_row, start_col, end_row, end_col, board[start_row][start_col]):
        if end_col == 6:
            board[start_row][start_col] = ' '
            board[start_row][end_col] = 'K' if board[start_row][start_col].isupper() else 'k'
            if board[start_row][start_col].isupper():
                board[start_row][7] = ' '
                board[start_row][5] = 'R'
            else:
                board[start_row][0] = ' '
                board[start_row][3] = 'r'
        elif end_col == 2:
            board[start_row][start_col] = ' '
            board[start_row][end_col] = 'K' if board[start_row][start_col].isupper() else 'k'
            if board[start_row][start_col].isupper():
                board[start_row][0] = ' '
                board[start_row][3] = 'R'
            else:
                board[start_row][7] = ' '
                board[start_row][5] = 'r'
        return

In [575]:
def get_clicked_square(mouse_pos):
    col = mouse_pos[0] // SQUARE_SIZE
    row = 7 - (mouse_pos[1] // SQUARE_SIZE)
    return row, col

In [576]:
def make_move(board, screen, start_row, start_col, end_row, end_col):
    piece = board[start_row][start_col]
    board[end_row][end_col] = piece
    board[start_row][start_col] = ' '

    if piece.lower() == 'k' and abs(end_col - start_col) == 2:
        if end_col > start_col:
            rook = board[start_row][7]
            board[start_row][7] = ' '
            board[start_row][5] = rook
        else:
            rook = board[start_row][0]
            board[start_row][0] = ' '
            board[start_row][3] = rook

    if piece.lower() == 'p' and (end_row == 0 or end_row == 7):
        board[end_row][end_col] = 'Q' if piece.isupper() else 'q'

    if piece.lower() == 'p' and abs(start_row - end_row) == 2:
        en_passant_possible = (end_row, end_col)
    else:
        en_passant_possible = None

    if piece.lower() == 'p' and start_col != end_col and board[end_row][end_col] == ' ':
        board[start_row][end_col] = ' '

    draw_board(screen)
    for row in range(DIMENSION):
        for col in range(DIMENSION):
            piece = board[row][col]
            if piece != ' ':
                piece_image_file = piece.upper() + ".png"
                place_piece(screen, piece_image_file, row, col)
    pygame.display.flip()

def main():
    current_player = 'b'
    
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Chess")

    clock = pygame.time.Clock()
    screen.fill((255, 255, 255))
    global en_passant_possible
    en_passant_possible = None
    running = True
    selected_piece = None

    print("Initial board state:")
    for row in board:
        print(row)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                mouse_pos = pygame.mouse.get_pos()
                row, col = get_clicked_square(mouse_pos)
                clicked_piece = board[row][col]
                print(f"Clicked square: {row}, {col}")
                print(f"Current player: {current_player}")
                print(f"Piece at clicked square: '{clicked_piece}'")
                
                if selected_piece is None:
                    if clicked_piece != ' ':
                        if (clicked_piece.isupper() and current_player == 'w') or (clicked_piece.islower() and current_player == 'b'):
                            selected_piece = (row, col)
                            print(f"Selected piece: {clicked_piece} at {row}, {col}")
                        else:
                            print(f"This piece ({clicked_piece}) belongs to the other player.")
                    else:
                        print("No piece at this square.")
                else:
                    start_row, start_col = selected_piece
                    end_row, end_col = row, col
                    if is_valid_move(board, start_row, start_col, end_row, end_col, current_player, en_passant_possible):
                        make_move(board, screen, start_row, start_col, end_row, end_col)
                        current_player = 'b' if current_player == 'w' else 'w'
                        print(f"Moved piece from {start_row}, {start_col} to {end_row}, {end_col}")
                        print("Updated board state:")
                        for row in board:
                            print(row)
                    else:
                        print(f"Invalid move from {start_row}, {start_col} to {end_row}, {end_col}")
                    selected_piece = None

        draw_board(screen)
        for row in range(DIMENSION):
            for col in range(DIMENSION):
                piece = board[row][col]
                if piece != ' ':
                    piece_image_file = piece.upper() + ".png"
                    place_piece(screen, piece_image_file, row, col)
        
        if selected_piece:
            row, col = selected_piece
            pygame.draw.rect(screen, (255, 255, 0), pygame.Rect(col * SQUARE_SIZE, (7-row) * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE), 3)

        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

Initial board state:
['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP']
['BR', 'BN', 'BB', 'BQ', 'BK', 'BB', 'BN', 'BR']
Clicked square: 1, 3
Current player: b
Piece at clicked square: 'p'
Selected piece: p at 1, 3
Clicked square: 3, 3
Current player: b
Piece at clicked square: ' '
Moved piece from 1, 3 to 3, 3
Updated board state:
['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r']
['p', 'p', 'p', ' ', 'p', 'p', 'p', 'p']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', 'p', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
['BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP', 'BP']
['BR', 'BN', 'BB', 'BQ', 'BK', 'BB', 'BN', 'BR']


SystemExit: 